In [2]:
pip install torch_scatter

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam

from torch_geometric.datasets import Planetoid
from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.nn.conv import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges

from typing import Optional
from torch_scatter import scatter_add
from torch_sparse import coalesce
from torch_geometric.utils import add_self_loops, remove_self_loops, to_scipy_sparse_matrix
from torch_geometric.utils.num_nodes import maybe_num_nodes
from scipy.sparse.linalg import eigsh

In [29]:
from torch_geometric.data import Data
data = Data(x = X, edge_index = edge_index)
data = train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1)
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [30]:
import torch

num_nodes = data.num_nodes
directed_edge_index = torch.empty((2, 0), dtype=torch.long)


In [31]:
for src, tgt in data.edge_index.t().tolist():
    # Determine the direction based on your problem context
    # For example, you can assign a direction based on the node IDs or node attributes
    if src < tgt:
        directed_edge_index = torch.cat((directed_edge_index, torch.tensor([[src], [tgt]])), dim=1)
    else:
        directed_edge_index = torch.cat((directed_edge_index, torch.tensor([[tgt], [src]])), dim=1)


In [33]:
directed_edge_index.shape

torch.Size([2, 10556])

In [34]:
data.edge_index = directed_edge_index


In [35]:
data = train_test_split_edges(data, 0.05, 0.1)
data

/Users/leoenoch/opt/anaconda3/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], val_pos_edge_index=[2, 527], test_pos_edge_index=[2, 1055], train_pos_edge_index=[2, 10330], train_neg_adj_mask=[2708, 2708], val_neg_edge_index=[2, 527], test_neg_edge_index=[2, 1055])